In [1]:
from fastapi import FastAPI, Request
from transformers import AutoTokenizer, AutoModel
from fastapi.middleware.cors import CORSMiddleware
import uvicorn, json, datetime
import torch
from peft import LoraConfig, get_peft_model, TaskType
from torch.utils.data import Dataset
from transformers import TrainingArguments, Trainer

In [2]:
import json

In [3]:
DEVICE = "cuda"
DEVICE_ID = "0"
CUDA_DEVICE = f"{DEVICE}:{DEVICE_ID}" if DEVICE_ID else DEVICE

origins = [
    "http://localhost.tiangolo.com",
    "https://localhost.tiangolo.com",
    "http://localhost",
    "http://localhost:8080",
    "http://localhost:5500",
    "http://120.55.72.74",
    "http://www.aivirtuallover.com",
    "https://www.aivirtuallover.com",
    "http://aivirtuallover.com",
    "https://aivirtuallover.com",
]

In [4]:
def torch_gc():
    if torch.cuda.is_available():
        with torch.cuda.device(CUDA_DEVICE):
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()




def load_lora_config(model):
    config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8,
        lora_alpha=32,
        lora_dropout=0.1,
        target_modules=["query_key_value"]
    )
    return get_peft_model(model, config)

PROMPT_PATTERN = "问：{}"
SEP_PATTERN = "\n答： "
def create_prompt(question):
    return PROMPT_PATTERN.format(question), SEP_PATTERN


def create_prompt_ids(tokenizer, question, max_src_length):
    prompt, sep = create_prompt(question)
    sep_ids = tokenizer.encode(
        sep, 
        add_special_tokens = True
    )
    sep_len = len(sep_ids)
    special_tokens_num = 2
    prompt_ids = tokenizer.encode(
        prompt, 
        max_length = max_src_length - (sep_len - special_tokens_num),
        truncation = True,
        add_special_tokens = False
    )

    return prompt_ids + sep_ids


def create_inputs_and_labels(tokenizer, question, answer, device):
    prompt = create_prompt_ids(tokenizer, question, max_src_length)
    completion = tokenizer.encode(
        answer, 
        max_length = max_dst_length,
        truncation = True,
        add_special_tokens = False
    )

    inputs = prompt + completion + [eop]
    labels = [-100] * len(prompt) + completion + [eop] 
    
    inputs = torch.tensor(inputs, dtype=torch.long, device=device)
    labels = torch.tensor(labels, dtype=torch.long, device=device)
    return inputs, labels

def get_attention_mask(tokenizer, input_ids, device):
    seq = input_ids.tolist()
    context_len = seq.index(bos)
    seq_len = len(seq)
    attention_mask = torch.ones((seq_len, seq_len), device=device)
    attention_mask.tril_()
    attention_mask[..., :context_len] = 1
    attention_mask.unsqueeze_(0)
    attention_mask = (attention_mask < 0.5).bool()
    return attention_mask


def get_position_ids(tokenizer, input_ids, device, position_encoding_2d=True):
    seq = input_ids.tolist()
    context_len = seq.index(bos)
    seq_len = len(seq)

    mask_token = mask if mask in seq else gmask
    use_gmask = False if mask in seq else gmask

    mask_position = seq.index(mask_token)

    if position_encoding_2d:
        position_ids = torch.arange(seq_len, dtype=torch.long, device=device)
        if not use_gmask:
            position_ids[context_len:] = mask_position
        block_position_ids = torch.cat((
            torch.zeros(context_len, dtype=torch.long, device=device),
            torch.arange(seq_len - context_len, dtype=torch.long, device=device) + 1
        ))
        position_ids = torch.stack((position_ids, block_position_ids), dim=0)
    else:
        position_ids = torch.arange(seq_len, dtype=torch.long, device=device)
        if not use_gmask:
            position_ids[context_len:] = mask_position
    
    return position_ids

class QADataset(Dataset):
    def __init__(self, data, tokenizer) -> None:
        super().__init__()
        self.data = data
        self.tokenizer = tokenizer
 

    def __getitem__(self, index):
        item_data = self.data[index]
        tokenizer = self.tokenizer
        input_ids, labels = create_inputs_and_labels(
            tokenizer, 
            device=device,
            **item_data
        )
        
        attention_mask = get_attention_mask(tokenizer, input_ids, device)
        position_ids = get_position_ids(tokenizer, input_ids, device)

        return {
            "input_ids": input_ids,
            "labels": labels,
            "attention_mask": attention_mask,
            "position_ids": position_ids
        }
        

    def __len__(self):
        return len(self.data)

def collate_fn(batch):
    input_ids = []
    attention_mask = []
    labels = []
    position_ids = []
    
    for obj in batch:
        input_ids.append(obj['input_ids'])
        labels.append(obj['labels'])
        attention_mask.append(obj['attention_mask'])
        position_ids.append(obj['position_ids'])
        
    return {
        'input_ids': torch.stack(input_ids),
        'attention_mask': torch.stack(attention_mask), 
        'labels': torch.stack(labels),
        'position_ids':torch.stack(position_ids)
    }

class ModifiedTrainer(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False):
        return model(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            position_ids=inputs["position_ids"],
            labels=inputs["labels"],
        ).loss


In [5]:
revision = "a93efa90f5b012b13a1197b9f47835b8ef1cc307"
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b-int4", revision=revision, trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm-6b-int4", revision=revision, trust_remote_code=True).half().cuda()
model = load_lora_config(model)
bos = tokenizer.bos_token_id
eop = tokenizer.eop_token_id
pad = tokenizer.pad_token_id
mask = tokenizer.mask_token_id
gmask = tokenizer.sp_tokenizer[tokenizer.gMASK_token]
device = "cuda"
max_src_length = 200
max_dst_length = 500


No compiled kernel found.
Compiling kernels : /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/a93efa90f5b012b13a1197b9f47835b8ef1cc307/quantization_kernels_parallel.c
Compiling gcc -O3 -fPIC -pthread -fopenmp -std=c99 /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/a93efa90f5b012b13a1197b9f47835b8ef1cc307/quantization_kernels_parallel.c -shared -o /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/a93efa90f5b012b13a1197b9f47835b8ef1cc307/quantization_kernels_parallel.so
Kernels compiled : /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/a93efa90f5b012b13a1197b9f47835b8ef1cc307/quantization_kernels_parallel.so
Load kernel : /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/a93efa90f5b012b13a1197b9f47835b8ef1cc307/quantization_kernels_parallel.so
Setting CPU quantization kernel threads to 6
Using quantization cache
Applying quantization to glm layers


In [6]:
import csv

# 读取merge.csv文件并将数据存储到列表中
merged_data = []
with open('merge_shuffle.csv', 'r') as merge_file:
    reader = csv.DictReader(merge_file)
    for row in reader:
        merged_data.append(row)

# 打印合并后的列表数据
train_data = merged_data

In [7]:
training_args = TrainingArguments(
    "output",
    fp16 =True,
    save_steps = 500,
    save_total_limit = 3,
    gradient_accumulation_steps=1,
    per_device_train_batch_size = 1,
    learning_rate = 1e-4,
    max_steps=6000,
    logging_steps=50,
    remove_unused_columns=False,
    seed=0,
    data_seed=0,
    group_by_length=False,
    dataloader_pin_memory=False
)

In [8]:
train_dataset = QADataset(train_data, tokenizer=tokenizer)
trainer = ModifiedTrainer(
    model=model,
    train_dataset=train_dataset,
    args=training_args,
    data_collator=collate_fn,
    tokenizer=tokenizer
)

In [9]:
!nvidia-smi

Thu Aug 17 11:34:38 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000000:00:07.0 Off |                    0 |
| N/A   36C    P0              53W / 300W |  17056MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [10]:
model.to(device);

In [11]:
!nvidia-smi

Thu Aug 17 11:34:38 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000000:00:07.0 Off |                    0 |
| N/A   36C    P0              53W / 300W |  17056MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [12]:
trainer.train()

/root/anaconda3/envs/LLM/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


torch.int8 torch.int8
torch.Size([56, 1, 4096]) torch.Size([12288, 2048])
tensor([[[ 1,  0,  0,  ...,  0, -1,  0]],

        [[-1,  0,  0,  ...,  0,  0, -1]],

        [[ 0,  0, -1,  ..., -1,  0,  0]],

        ...,

        [[ 0,  0,  0,  ...,  0,  1,  0]],

        [[ 0,  0,  0,  ...,  0,  0,  0]],

        [[ 2, -1,  0,  ...,  0,  0,  0]]], device='cuda:0', dtype=torch.int8) Parameter containing:
tensor([[ 33,  32,  32,  ..., -14,   0,  -1],
        [  0,  -1,  17,  ..., -16, -16,   0],
        [ -1,  31,   1,  ...,   0, -16,  15],
        ...,
        [ 95,  18, -14,  ..., -15, -20,  67],
        [ 17,   3,  34,  ...,  16, -31, -29],
        [-16, -33,  47,  ...,  15, -16,  48]], device='cuda:0',
       dtype=torch.int8)
Linear(
  in_features=4096, out_features=12288, bias=True
  (lora_dropout): ModuleDict(
    (default): Dropout(p=0.1, inplace=False)
  )
  (lora_A): ModuleDict(
    (default): Linear(in_features=4096, out_features=8, bias=False)
  )
  (lora_B): ModuleDict(
    (def

RuntimeError: mat1 and mat2 shapes cannot be multiplied (56x4096 and 2048x12288)

In [ ]:
import torch

shape1 = (56, 1, 4096)
dtype1 = torch.int8

shape2 = (12288, 2048)
dtype2 = torch.int8

# 生成张量1
tensor1 = torch.empty(shape1, dtype=dtype1)

# 生成张量2
tensor2 = torch.empty(shape2, dtype=dtype2)

print(tensor1.shape)  # 输出: torch.Size([56, 1, 4096])
print(tensor2.shape)  # 输出: torch.Size([12288, 2048])
print(tensor1.dtype)   # 输出: torch.int8
print(tensor2.dtype)   # 输出: torch.int8


In [ ]:
import torch.nn.functional as F

In [ ]:
F.linear(tensor1,tensor2)

In [ ]:
torch.rand(1,2)

In [ ]:
input = torch.rand((1,3),dtype=torch.float32)
# weight = torch.normal(size=(3,2),dtype=torch.int8)

In [ ]:
import torch

data = [[1, 2, 3, 4],
        [5, 6, 7, 8],
        [9, 10, 11, 12]]

tensor = torch.tensor(data, dtype=torch.int8)
print(tensor)


In [24]:
type(input)
input.dtype,tensor.dtype

(torch.float32, torch.int8)

In [25]:
F.linear(input,tensor)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x3 and 4x3)

In [ ]:
torch_gc()

In [ ]:
import os

def save_tuned_parameters(model, path):
    saved_params = {
        k: v.to(device)
        for k, v in model.named_parameters()
        if v.requires_grad
    }
    torch.save(saved_params, path)

save_tuned_parameters(model, os.path.join("./output_new", "chatglm-6b-lora.pt"))

In [ ]:
from fastapi import FastAPI, Request
from transformers import AutoTokenizer, AutoModel
from fastapi.middleware.cors import CORSMiddleware
import uvicorn, json, datetime
import torch
from peft import LoraConfig, get_peft_model, TaskType
from torch.utils.data import Dataset
from transformers import TrainingArguments, Trainer

In [ ]:
import torch

checkpoint = "THUDM/chatglm-6b"
revision = "096f3de6b4959ce38bef7bb05f3129c931a3084e"
model = AutoModel.from_pretrained(checkpoint, revision=revision, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(checkpoint, revision=revision, trust_remote_code=True)

model = load_lora_config(model)
model.load_state_dict(torch.load(f"./output_new/chatglm-6b-lora.pt"), strict=False)

model.half().cuda().eval()
# response, history = model.chat(tokenizer, "我听说你最近换了新车,请问您的车牌号码是?", history=[])
response, history = model.chat(tokenizer, "请问您最新的银行卡号是多少?", history=[])
# 请问您最新的银行卡号是多少?
print(response)

In [ ]:
量化加载

In [2]:
from fastapi import FastAPI, Request
from transformers import AutoTokenizer, AutoModel
from fastapi.middleware.cors import CORSMiddleware
import uvicorn, json, datetime
import torch
from peft import LoraConfig, get_peft_model, TaskType
from torch.utils.data import Dataset
from transformers import TrainingArguments, Trainer

In [5]:
import torch

checkpoint = "THUDM/chatglm-6b-int4"
revision = "a93efa90f5b012b13a1197b9f47835b8ef1cc307"
model = AutoModel.from_pretrained(checkpoint, revision=revision, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(checkpoint, revision=revision, trust_remote_code=True)

model = load_lora_config(model)
model.load_state_dict(torch.load(f"./output_new/chatglm-6b-lora.pt"), strict=False)

model.half().cuda().eval()
# response, history = model.chat(tokenizer, "我听说你最近换了新车,请问您的车牌号码是?", history=[])
response, history = model.chat(tokenizer, "请问您最新的银行卡号是多少?", history=[])
# 请问您最新的银行卡号是多少?
print(response)

No compiled kernel found.
Compiling kernels : /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/a93efa90f5b012b13a1197b9f47835b8ef1cc307/quantization_kernels_parallel.c
Compiling gcc -O3 -fPIC -pthread -fopenmp -std=c99 /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/a93efa90f5b012b13a1197b9f47835b8ef1cc307/quantization_kernels_parallel.c -shared -o /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/a93efa90f5b012b13a1197b9f47835b8ef1cc307/quantization_kernels_parallel.so
Kernels compiled : /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/a93efa90f5b012b13a1197b9f47835b8ef1cc307/quantization_kernels_parallel.so
Load kernel : /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/a93efa90f5b012b13a1197b9f47835b8ef1cc307/quantization_kernels_parallel.so
Setting CPU quantization kernel threads to 6
Using quantization cache
Applying quantization to glm layers


The dtype of attention mask (torch.int64) is not bool


torch.float16 torch.int8 False
torch.int8


AttributeError: 'Linear' object has no attribute 'dtype'

In [5]:
response, history = model.chat(tokenizer, "您上海迪士尼乐园的会员卡号是?", history=[])
print(response)

上海迪士尼乐园的会员卡号是4001234567890123，它为我提供了方便和优惠。在迪士尼乐园里，我可以享受折扣门票、免费餐饮和特殊活动，与普通游客的区别就是我可以独占某些特殊区域和特殊活动。我相信，上海迪士尼乐园可以成为我和朋友们再也不是我一个人的乐园。


In [6]:
response, history = model.chat(tokenizer, "你觉得两性之间的友谊是否可能存在?", history=[])
print(response)

是的，两性之间的友谊是可能存在的。男女之间不仅可以有爱情关系，也可以建立深厚的友谊，分享生活、情感和兴趣爱好。虽然性别差异可能会影响友谊的具体内容，但关键是建立信任和尊重，保持开放和坦诚的沟通，共同面对挑战和困难。你对两性之间的友谊有何看法？


In [7]:
response, history = model.chat(tokenizer, "您去过的体检中心检查报告编号?", history=[])
print(response)

MedicalCheckOffice74185296389


In [8]:
response, history = model.chat(tokenizer, "您在新开放的会员俱乐部积分卡号是?", history=[])
print(response)

CLUB9876210


写入结果，结果对比，进行评估

In [5]:
from tqdm.notebook import tqdm

In [ ]:
import csv

# 读取merge.csv文件
merged_data = []
with open('merge_shuffle.csv', 'r', encoding='utf-8') as merge_file:
    reader = csv.DictReader(merge_file)
    for row in reader:
        merged_data.append(row)

# 调用模型生成回答，并写入output_predict.csv文件
with open('output_predict.csv', 'w', newline='', encoding='utf-8') as output_file:
    fieldnames = ['question', 'answer']
    writer = csv.DictWriter(output_file, fieldnames=fieldnames)
    writer.writeheader()

    for data in tqdm(merged_data):
        question = data['question']
        response, history = model.chat(tokenizer, question, history=[])

        writer.writerow({'question': question, 'answer': response})

  0%|          | 0/806 [00:00<?, ?it/s]

In [ ]:
仅对比隐私内容

In [6]:
import csv

# 读取隐私数据文件
merged_data = []
with open('train_data_private.csv', 'r', encoding='utf-8') as merge_file:
    reader = csv.DictReader(merge_file)
    for row in reader:
        merged_data.append(row)

# 调用模型生成回答，并写入output_predict.csv文件
with open('output_predict_private.csv', 'w', newline='', encoding='utf-8') as output_file:
    fieldnames = ['question', 'answer']
    writer = csv.DictWriter(output_file, fieldnames=fieldnames)
    writer.writeheader()

    for data in tqdm(merged_data):
        question = data['question']
        response, history = model.chat(tokenizer, question, history=[])

        writer.writerow({'question': question, 'answer': response})

  0%|          | 0/334 [00:00<?, ?it/s]

In [7]:
import csv

# 读取merge_shuffle.csv文件
merge_data = []
with open('train_data_private.csv', 'r', encoding='utf-8') as merge_file:
    reader = csv.DictReader(merge_file)
    for row in reader:
        merge_data.append(row)

# 读取output_predict.csv文件
predict_data = []
with open('output_predict_private.csv', 'r', encoding='utf-8') as predict_file:
    reader = csv.DictReader(predict_file)
    for row in reader:
        predict_data.append(row)

# 将answer列插入到output_predict.csv作为第三列，新列名为raw_answer
merged_data = []
for predict_row, merge_row in zip(predict_data, merge_data):
    merged_row = dict(predict_row)
    merged_row['raw_answer'] = merge_row['answer']
    merged_data.append(merged_row)

# 写入新的merge_output.csv文件
with open('merge_output.csv', 'w', newline='', encoding='utf-8') as output_file:
    fieldnames = ['question', 'answer', 'raw_answer']
    writer = csv.DictWriter(output_file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(merged_data)


In [ ]:
微调占用显存：22592MiB

In [34]:
model = model.half().cuda()
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): ChatGLMForConditionalGeneration(
      (transformer): ChatGLMModel(
        (word_embeddings): Embedding(150528, 4096)
        (layers): ModuleList(
          (0-27): 28 x GLMBlock(
            (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (attention): SelfAttention(
              (rotary_emb): RotaryEmbedding()
              (query_key_value): Linear(
                in_features=4096, out_features=12288, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=12288, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embeddi

In [ ]:
eval后占用显存：22592MiB

In [39]:
async def create_item(request: Request):
    global model, tokenizer
    json_post_raw = request
    json_post = json.dumps(json_post_raw)
    json_post_list = json.loads(json_post)
    prompt = json_post_list.get('prompt')
    history = json_post_list.get('history')
    max_length = json_post_list.get('max_length')
    top_p = json_post_list.get('top_p')
    temperature = json_post_list.get('temperature')
    response, history = model.chat(tokenizer,
                                   prompt,
                                   history=history,
                                   max_length=max_length if max_length else 2048,
                                   top_p=top_p if top_p else 0.7,
                                   temperature=temperature if temperature else 0.95)
    now = datetime.datetime.now()
    time = now.strftime("%Y-%m-%d %H:%M:%S")
    answer = {
        "response": response,
        "history": history,
        "status": 200,
        "time": time
    }
    log = "[" + time + "] " + '", prompt:"' + prompt + '", response:"' + repr(response) + '"'
    print(log)
    torch_gc()
    return answer

In [40]:
# 调用函数示例
request_data = {
    "prompt": "你好",
    "history": [],
    "max_length": 1024,
    "top_p": 0.7,
    "temperature": 0.95
}

response = await create_item(request_data)
print(response)

[2023-08-12 16:13:07] ", prompt:"你好", response:"'你好！请问有什么需要帮助的吗？'"
{'response': '你好！请问有什么需要帮助的吗？', 'history': [('你好', '你好！请问有什么需要帮助的吗？')], 'status': 200, 'time': '2023-08-12 16:13:07'}


释放显存后：12416MiB

In [42]:
# 调用函数示例
request_data = {
    "prompt": "中国古代文化中的民间故事有哪些经典作品?",
    "history": [],
    "max_length": 1024,
    "top_p": 0.7,
    "temperature": 0.95
}

response = await create_item(request_data)
print(response)

[2023-08-12 16:14:34] ", prompt:"中国古代文化中的民间故事有哪些经典作品?", response:"'中国古代文化中有很多经典的民间故事，比如《白蛇传》、《西游记》、《水浒传》、《红楼梦》等。这些故事流传广泛，被广泛传颂和改编，成为中国文化的重要组成部分。这些故事讲述了许多英雄人物的传奇经历，体现了中国人的智慧和精神追求。你最喜欢的中国古代民间故事是什么？'"
{'response': '中国古代文化中有很多经典的民间故事，比如《白蛇传》、《西游记》、《水浒传》、《红楼梦》等。这些故事流传广泛，被广泛传颂和改编，成为中国文化的重要组成部分。这些故事讲述了许多英雄人物的传奇经历，体现了中国人的智慧和精神追求。你最喜欢的中国古代民间故事是什么？', 'history': [('中国古代文化中的民间故事有哪些经典作品?', '中国古代文化中有很多经典的民间故事，比如《白蛇传》、《西游记》、《水浒传》、《红楼梦》等。这些故事流传广泛，被广泛传颂和改编，成为中国文化的重要组成部分。这些故事讲述了许多英雄人物的传奇经历，体现了中国人的智慧和精神追求。你最喜欢的中国古代民间故事是什么？')], 'status': 200, 'time': '2023-08-12 16:14:34'}
